In [ ]:
import yaml
import random
import numpy as np
import torch
from transformers import AutoTokenizer, AutoProcessor
from dataset.dataset import TsQaDataset, DataCollator
from models.TimeLanguageModel import TLM, TLMConfig

/opt/conda/envs/time_qa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [2]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
with open('yaml/qwen7B_infer.yaml', 'r') as f:
    config = yaml.safe_load(f)
print("🛠️ 加载配置文件内容:")
for k, v in config.items():
    print(f"  {k}: {v}")

class ConfigObject:
    """递归将字典转换为嵌套对象"""
    def __init__(self, data):
        for key, value in data.items():
            # 递归处理嵌套字典
            if isinstance(value, dict):
                setattr(self, key, ConfigObject(value))
            # 直接赋值非字典类型
            else:
                setattr(self, key, value)
args=ConfigObject(config)

🛠️ 加载配置文件内容:
  model: TimeSeriesEncoder
  d_model: 512
  n_heads: 8
  e_layers: 4
  patch_len: 60
  stride: 60
  input_len: 600
  dropout: 0.1
  tt_d_model: 896
  tt_n_heads: 16
  tt_layers: 2
  tt_dropout: 0.1
  prefix_num: 25
  llm_model_path: /inspire/hdd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/weizhongyu-24036/time_series/Time-QA-new/LLM/Qwen2.5-7B-Instruct
  pretrain: False
  min_mask_ratio: 0.7
  max_mask_ratio: 0.8
  ts_path_test: dataset/dataset_processing/data_merged_new.h5
  qa_path_test: dataset/dataset_processing/test_sw3000.jsonl
  do_train: True
  per_device_train_batch_size: 6
  per_device_eval_batch_size: 6
  learning_rate: 3e-05
  gradient_accumulation_steps: 1
  num_train_epochs: 2
  weight_decay: 1e-06
  freeze_ts_model: True
  fp16: True
  dataloader_pin_memory: True
  dataloader_num_workers: 4


In [4]:
print(type(args))

<class '__main__.ConfigObject'>


In [5]:
tlmconfig = TLMConfig(
    llm_model_path=config['llm_model_path'],
    freeze_ts_model=config['freeze_ts_model'],
    ts_pad_num=config['prefix_num']
)

In [6]:
tlmconfig

TLMConfig {
  "freeze_ts_model": true,
  "llm_model_path": "/inspire/hdd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/weizhongyu-24036/time_series/Time-QA-new/LLM/Qwen2.5-7B-Instruct",
  "model_type": "vlm_model",
  "transformers_version": "4.47.1",
  "ts_pad_num": 25
}

In [7]:
model = TLM.from_pretrained('/inspire/hdd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/weizhongyu-24036/time_series/Time-QA-new/save/sft_v2_5epoch_qwen_7B/checkpoint-6346', config=tlmconfig, ts_config=args).cuda()

Loading checkpoint shards: 100%|██████████| 7/7 [00:01<00:00,  5.03it/s]


🧊 Completely freezing ALL LLM parameters
🧊 Frozen: model.embed_tokens.weight
🧊 Frozen: lm_head.weight
🧊 Freezing Time Series Encoder
🔥 itformer: 47,908,224 trainable parameters
🔥 ts_project: 459,648 trainable parameters
🔥 query_project: 3,212,160 trainable parameters
🔥 fusion_project: 3,214,848 trainable parameters

📊 详细参数统计:
总参数: 7,687,285,948
可训练参数: 54,794,880
可训练比例: 0.71%

📋 各模块详情:
🧊 llm_model      :          0 / 7,615,616,512 (  0.0%)
🧊 ts_encoder     :          0 / 16,874,556 (  0.0%)
🔥 itformer       : 47,908,224 / 47,908,224 (100.0%)
🔥 ts_project     :    459,648 /    459,648 (100.0%)
🔥 query_project  :  3,212,160 /  3,212,160 (100.0%)
🔥 fusion_project :  3,214,848 /  3,214,848 (100.0%)


Loading checkpoint shards: 100%|██████████| 7/7 [00:01<00:00,  5.09it/s]


In [8]:
model

TLM(
  (llm_model): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(152064, 3584)
      (layers): ModuleList(
        (0-27): 28 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
            (k_proj): Linear(in_features=3584, out_features=512, bias=True)
            (v_proj): Linear(in_features=3584, out_features=512, bias=True)
            (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
            (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
            (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
          (post_attention_layernorm): 

In [9]:
model.eval()

TLM(
  (llm_model): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(152064, 3584)
      (layers): ModuleList(
        (0-27): 28 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
            (k_proj): Linear(in_features=3584, out_features=512, bias=True)
            (v_proj): Linear(in_features=3584, out_features=512, bias=True)
            (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
            (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
            (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
          (post_attention_layernorm): 

In [10]:
print(f"✅ 模型加载完成! 参数量: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

✅ 模型加载完成! 参数量: 7687.29M


In [11]:
tokenizer = AutoTokenizer.from_pretrained(tlmconfig.llm_model_path)
tokenizer.padding_side = 'left'
processor = AutoProcessor.from_pretrained(tlmconfig.llm_model_path)

In [12]:
test_dataset = TsQaDataset(
    config['ts_path_test'],
    config['qa_path_test'],
    tokenizer,
    processor,
    tlmconfig
)

📊 Vocab size: 151665
🔍 验证特殊token:
✅ pad_token_id = 151643
✅ eos_token_id = 151645


In [13]:
print(len(test_dataset))

42477


In [24]:
random_index = random.randint(1, len(test_dataset)) 
sample = test_dataset[random_index]
sample


{'form': 'open',
 'stage': 1,
 'query_ids': [641,
  279,
  2266,
  315,
  279,
  3897,
  4712,
  8286,
  11,
  1128,
  374,
  279,
  23560,
  13042,
  323,
  24586,
  25361,
  315,
  279,
  2297,
  304,
  10657,
  37022,
  518,
  279,
  12041,
  12,
  68269,
  1198,
  56220,
  320,
  43,
  4872,
  8,
  26389,
  13166,
  2337,
  264,
  3175,
  10775,
  5267,
  366,
  2576,
  29],
 'input_ids': [151644,
  8948,
  198,
  2610,
  525,
  264,
  10950,
  17847,
  13,
  151645,
  198,
  151644,
  872,
  198,
  641,
  279,
  2266,
  315,
  279,
  3897,
  4712,
  8286,
  11,
  1128,
  374,
  279,
  23560,
  13042,
  323,
  24586,
  25361,
  315,
  279,
  2297,
  304,
  10657,
  37022,
  518,
  279,
  12041,
  12,
  68269,
  1198,
  56220,
  320,
  43,
  4872,
  8,
  26389,
  13166,
  2337,
  264,
  3175,
  10775,
  5267,
  220,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,
  151655,

In [25]:
for key, value in sample.items():
    if torch.is_tensor(value):
        print(f"  {key}: 形状 {value.shape}, dtype {value.dtype}")
    else:
        print(f"  {key}: {value}")

  form: open
  stage: 1
  query_ids: [641, 279, 2266, 315, 279, 3897, 4712, 8286, 11, 1128, 374, 279, 23560, 13042, 323, 24586, 25361, 315, 279, 2297, 304, 10657, 37022, 518, 279, 12041, 12, 68269, 1198, 56220, 320, 43, 4872, 8, 26389, 13166, 2337, 264, 3175, 10775, 5267, 366, 2576, 29]
  input_ids: [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 641, 279, 2266, 315, 279, 3897, 4712, 8286, 11, 1128, 374, 279, 23560, 13042, 323, 24586, 25361, 315, 279, 2297, 304, 10657, 37022, 518, 279, 12041, 12, 68269, 1198, 56220, 320, 43, 4872, 8, 26389, 13166, 2337, 264, 3175, 10775, 5267, 220, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151655, 151645, 198, 151644, 77091, 198]
  labels: [785, 10657, 37022, 518, 279, 49075, 26389, 8458, 20699, 15175, 13, 1096, 19753, 77764, 264, 14720, 323, 24020, 58877, 5068, 11, 44

In [26]:
data_collator = DataCollator(tokenizer=tokenizer)
collated = data_collator([sample])
print(f"Collated keys: {list(collated.keys())}")

Collated keys: ['input_ids', 'attention_mask', 'labels', 'ts_values', 'stage', 'index', 'query_ids']


In [27]:
input_ids = collated['input_ids'].cuda()
ts_values = collated['ts_values'].cuda()
attention_mask = collated['attention_mask'].cuda()
query_ids = collated['query_ids'].cuda()
stages = collated['stage'].cuda()

In [28]:
type(collated['ts_values'])

torch.Tensor

In [29]:
raw_text = tokenizer.decode(input_ids[0].cpu().numpy(), skip_special_tokens=True)
raw_text

'system\nYou are a helpful assistant.\nuser\nIn the context of the provided engine signal, what is the precise representation and operational significance of the change in Total Temperature at the Low-Pressure Compressor (LPC) outlet observed during a single cycle?\n \nassistant\n'

In [30]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=input_ids,
        query_ids=query_ids,
        ts_values=ts_values,
        stage=stages,
        attention_mask=attention_mask,
        max_new_tokens=128,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.7,
        num_beams=1
    )

In [31]:
decoded_full = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
prediction = decoded_full.split('assistant\n')[-1] if 'assistant\n' in decoded_full else decoded_full
print(decoded_full)
print(prediction)

system
You are a helpful assistant.
user
In the context of the provided engine signal, what is the precise representation and operational significance of the change in Total Temperature at the Low-Pressure Compressor (LPC) outlet observed during a single cycle?
 
assistant
The Total Temperature at the LPC outlet remains constant throughout the cycle, indicating stable operational conditions.
-threats_to_the_environment
The Total Temperature at the LPC outlet remains constant throughout the cycle, indicating stable operational conditions.
-threats_to_the_environment


In [32]:
label_text = tokenizer.batch_decode(collated['labels'], skip_special_tokens=True)[0]
label_text

'The Total Temperature at the LPC outlet remains consistently stable. This stability signifies a reliable and steady compressor performance, demonstrating the absence of significant temperature fluctuations throughout the cycle.'